# Scikit-Learnを用いたノートブック上での特徴量変換
## 特徴量をSageMaker Feature Storeに保存

このノートブックでは、生のテキストをBERT埋め込みに変換します。
これにより、テキスト分類などの自然言語処理タスクを実行することができます。
この特徴量をSageMaker Feature Storeに保存します。

![](img/prepare_dataset_bert.png)

# BERT Mania!

![BERT Mania](img/bert_mania.png)

# BERT埋め込みを理解

* Bidirectional Encoder Representations from Transformers [BERT](https://arxiv.org/abs/1810.04805)
* Transformerアーキテクチャーの詳細については [Attention Is All You Need](https://arxiv.org/abs/1706.03762) を参照してください。

<img src="img/bert_embeddings.png" width="60%" align="left">

<img src="img/bert_input_features.png" width="80%" align="left">

* **input_ids**: 
事前学習済みBERTの語彙のID。トークンを表す。（トークンの数が `max_seq_length` より少ない場合は、0がパディングされる。）

* **input_mask**: 
BERTが注目すべきトークンを（0または1で）指定する。input_id のパディング部分には、ベクトル要素のそれぞれに 0 を与える。

* **segment_ids**:
セグメントIDは、テキスト分類のような単一シーケンスのタスクでは常に 0 となる。質問回答や次文予測などの2シーケンスのタスクでは 1 も使用される。
  
* **label_id**: 
各トレーニングデータの行のラベル（star_rating 1〜5）

In [ ]:
import sagemaker
import boto3

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='dsoaws/1.0'
)

# BERTの最大シーケンス長を定義

ここでは、最大シーケンス長はレビューテキストの単語数分布に基づいて選択します。

![](img/max_seq_length_viz.png)

In [ ]:
max_seq_length = 64

# Hugging FaceとTensorFlowを用いて生テキストをBERT特徴量に変換

In [ ]:
import tensorflow as tf
import collections
import json
import os
import pandas as pd
import csv
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

REVIEW_BODY_COLUMN = "review_body"
REVIEW_ID_COLUMN = "review_id"

LABEL_COLUMN = "star_rating"
LABEL_VALUES = [1, 2, 3, 4, 5]

label_map = {}
for (i, label) in enumerate(LABEL_VALUES):
    label_map[label] = i


class InputFeatures(object):
    """BERT特徴量ベクトル"""

    def __init__(self, input_ids, input_mask, segment_ids, label_id, review_id, date, label):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id
        self.review_id = review_id
        self.date = date
        self.label = label


class Input(object):
    """シーケンス分類で用いるトレーニング/テストの単一の入力"""

    def __init__(self, text, review_id, date, label=None):
        """入力のコンストラクタ
        Args:
          text: 文字列。トークン化されていない一つ目のシーケンスのテキスト。
            単一シーケンスのタスクではこのシーケンスのみを指定する。
          label: (オプショナル) 文字列。サンプルのラベル。トレーニングや検証用のサンプルでは指定する。
            テスト用のサンプルでは指定しない。
        """
        self.text = text
        self.review_id = review_id
        self.date = date
        self.label = label


def convert_input(the_input, max_seq_length):
    # まず、BERTが学習したデータ形式と合うようにデータを前処理する。
    # 1. テキストを小文字にする（BERT lowercaseモデルを用いる場合）
    # 2. トークン化する（例、"sally says hi" -> ["sally", "says", "hi"]）
    # 3. 単語をWordPieceに分割（例、"calling" -> ["call", "##ing"]）
    #
    # この辺りの処理はTransformersライブラリのトークナイザーがまかなってくれます。

    tokens = tokenizer.tokenize(the_input.text)
    tokens.insert(0, '[CLS]')
    tokens.append('[SEP]')
    print("**{} tokens**\n{}\n".format(len(tokens), tokens))

    encode_plus_tokens = tokenizer.encode_plus(
        the_input.text,
        pad_to_max_length=True,
        max_length=max_seq_length,
        truncation=True
    )
    
    # 事前学習済みBERTの語彙ID。トークンを表す。（トークン数が `max_seq_length` 未満であれば0をパディングする）
    input_ids = encode_plus_tokens["input_ids"]

    # BERTがどのトークンに注目するかを0/1で指定。`input_ids` のパディング部分のベクトル要素には0を割り当てる。
    input_mask = encode_plus_tokens["attention_mask"]

    # テキスト分類のような単一シーケンスのタスクではセグメントIDは常に0とする。質問回答や次文予測のような2シーケンスタスクの場合は1を割り当てる。
    segment_ids = [0] * max_seq_length

    # それぞれのトレーニングデータの行のラベル（`star_rating` 1〜5）
    label_id = label_map[the_input.label]

    features = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_id=label_id,
        review_id=the_input.review_id,
        date=the_input.date,
        label=the_input.label,
    )

    print("**{} input_ids**\n{}\n".format(len(features.input_ids), features.input_ids))
    print("**{} input_mask**\n{}\n".format(len(features.input_mask), features.input_mask))
    print("**{} segment_ids**\n{}\n".format(len(features.segment_ids), features.segment_ids))
    print("**label_id**\n{}\n".format(features.label_id))
    print("**review_id**\n{}\n".format(features.review_id))
    print("**date**\n{}\n".format(features.date))
    print("**label**\n{}\n".format(features.label))

    return features



def transform_inputs_to_tfrecord(inputs, output_file, max_seq_length):
    # データをBERTが理解できるフォーマットに変換する
    records = []
    tf_record_writer = tf.io.TFRecordWriter(output_file)

    for (input_idx, the_input) in enumerate(inputs):
        if input_idx % 10000 == 0:
            print("Writing input {} of {}\n".format(input_idx, len(inputs)))

        features = convert_input(the_input, max_seq_length)

        all_features = collections.OrderedDict()

        # input_ids、input_mask、segment_ids、label_idsを含んだTFRecordを作成
        all_features["input_ids"] = tf.train.Feature(int64_list=tf.train.Int64List(value=features.input_ids))
        all_features["input_mask"] = tf.train.Feature(int64_list=tf.train.Int64List(value=features.input_mask))
        all_features["segment_ids"] = tf.train.Feature(int64_list=tf.train.Int64List(value=features.segment_ids))
        all_features["label_ids"] = tf.train.Feature(int64_list=tf.train.Int64List(value=[features.label_id]))

        tf_record = tf.train.Example(features=tf.train.Features(feature=all_features))
        tf_record_writer.write(tf_record.SerializeToString())

        # Feature Storeに格納する、すべての特徴量を含んだレコードを作成
        records.append(
            {
                "input_ids": features.input_ids,
                "input_mask": features.input_mask,
                "segment_ids": features.segment_ids,
                "label_id": features.label_id,
                "review_id": the_input.review_id,
                "date": the_input.date,
                "label": features.label,
            }
        )

    tf_record_writer.close()

    return records

BERTの処理の準備としての特徴量エンジニアリングのフェーズで、それぞれの生のレビュー文（`review_body`）から3つの特徴量ベクトルが作成されます。

* **input_ids**: 
事前学習済みBERTの語彙のID。トークンを表す。（トークンの数が `max_seq_length` より少ない場合は、0がパディングされる。）

* **input_mask**: 
BERTが注目すべきトークンを（0または1で）指定する。input_id のパディング部分には、ベクトル要素のそれぞれに 0 を与える。

* **segment_ids**:
セグメントIDは、テキスト分類のような単一シーケンスのタスクでは常に 0 となる。質問回答や次文予測などの2シーケンスのタスクでは 1 も使用される。

また、それぞれの生のレビューデータから、1つのラベル（`star_rating`）が作成されます。

* **label_id**: 
各トレーニングデータの行のラベル（star_rating 1〜5）

# BERTに特化した特徴量エンジニアリングステップのデモ

ここでは、少量のデータを使ってコードのデモを行っていますが、後ほど、強力なSageMakerクラスター上でより多くのデータにスケールします。

## Feature Storeにはイベント時刻の特徴量が必要

レコード識別子名とイベント時刻の特徴量の名前が必要です。
これらを、データ内の対応する特徴量のカラムに対応させます。

注: イベント時刻の型は、Fractional（秒単位のUnixタイムスタンプ）、または、String（ISO-8601フォーマット）のいずれかでなければなりません。

In [ ]:
from datetime import datetime
from time import strftime

# timestamp = datetime.now().replace(microsecond=0).isoformat()
timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
print(timestamp)

In [ ]:
import pandas as pd

data = [
    [
        5,
        "ABCD12345",
        """I needed an "antivirus" application and know the quality of Norton products.  This was a no brainer for me and I am glad it was so simple to get.""",
    ],
    [
        3,
        "EFGH12345",
        """The problem with ElephantDrive is that it requires the use of Java. Since Java is notorious for security problems I haveit removed from all of my computers. What files I do have stored are photos.""",
    ],
    [
        1,
        "IJKL2345",
        """Terrible, none of my codes worked, and I can't uninstall it.  I think this product IS malware and viruses""",
    ],
]

df = pd.DataFrame(data, columns=["star_rating", "review_id", "review_body"])

# Input クラスを使用して、データからサンプルを作成する。
inputs = df.apply(
    lambda x: Input(label=x[LABEL_COLUMN], text=x[REVIEW_BODY_COLUMN], review_id=x[REVIEW_ID_COLUMN], date=timestamp),
    axis=1,
)

In [ ]:
# date が Feature Store の仕様に合わせて ISO-8601 になっていることを確認
print(inputs[0].date)

## TFRecord を保存

3つの特徴ベクトルと1つのラベルは、`TFRecord` インスタンスのリストに変換されます（トレーニングデータの各行に1つずつ）。
* **`tf_records`**:  トレーニングデータの各行のバイナリ表現（3つの特徴 + 1つのラベル）

これらの `TFRecord` は、パイプラインの残りの部分で使用されるエンジニアリング済みの特徴量です。

In [ ]:
output_file = "./data-tfrecord-featurestore/data.tfrecord"

# SageMaker Feature Store に特徴量を追加

## FeatureGroup を作成

特徴量グループ（Feature Group）とは、Feature Store で定義される、特徴量を論理的にまとめたものです。特徴量グループの定義は、特徴量の定義のリスト、レコードの識別子名、オンラインストアとオフラインストアの設定などで構成されます。

特徴量グループの管理には、create、describe、update、delete、list の各 API を使用できます。

In [ ]:
from time import gmtime, strftime, sleep

feature_group_name = "reviews-feature-group-" + strftime("%d-%H-%M-%S", gmtime())
print(feature_group_name)

In [ ]:
from sagemaker.feature_store.feature_definition import (
    FeatureDefinition,
    FeatureTypeEnum,
)

feature_definitions = [
    FeatureDefinition(feature_name="input_ids", feature_type=FeatureTypeEnum.STRING),
    FeatureDefinition(feature_name="input_mask", feature_type=FeatureTypeEnum.STRING),
    FeatureDefinition(feature_name="segment_ids", feature_type=FeatureTypeEnum.STRING),
    FeatureDefinition(feature_name="label_id", feature_type=FeatureTypeEnum.INTEGRAL),
    FeatureDefinition(feature_name="review_id", feature_type=FeatureTypeEnum.STRING),
    FeatureDefinition(feature_name="date", feature_type=FeatureTypeEnum.STRING),
    FeatureDefinition(feature_name="label", feature_type=FeatureTypeEnum.INTEGRAL),
    FeatureDefinition(feature_name="split_type", feature_type=FeatureTypeEnum.STRING),
]

In [ ]:
from sagemaker.feature_store.feature_group import FeatureGroup

feature_group = FeatureGroup(name=feature_group_name, feature_definitions=feature_definitions, sagemaker_session=sess)
print(feature_group)

## `record identifier` と `event time` の特徴量を指定

In [ ]:
record_identifier_feature_name = "review_id"
event_time_feature_name = "date"

## オフライン特徴量ストア用にS3プレフィックスをセット

In [ ]:
prefix = "reviews-feature-store-" + timestamp
print(prefix)

## Feature Group を作成

機能グループを作成するための最後のステップは、`create` メソッドを使用することです。
オンラインストアはデフォルトでは作成されないので、有効にしたい場合は `enable_online_store=True` とする必要があります。
`s3_uri`にはオフラインストアの場所を指定します。

In [ ]:
feature_group.create(
    s3_uri=f"s3://{bucket}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=False,
)

## Feature Group の詳細を確認

In [ ]:
feature_group.describe()

## Feature Store に取り込むレコードをレビュー

In [ ]:
records = transform_inputs_to_tfrecord(inputs, output_file, max_seq_length)

# _^^ ここ ^^の警告は無視してください_

## Feature Group の作成が完了するまでお待ちください

## _注意:  完了まで数分かかります。しばらくお待ちください。_

特徴量グループの作成は、データの読み込みに時間がかかります。
作成されてからでないと使用できません。
ステータスの確認は、以下の方法で行うことができます。

In [ ]:
import time


def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")

In [ ]:
wait_for_feature_group_creation_complete(feature_group=feature_group)

# Feature Store にレコードを取り込む

FeatureGroup が作成されたら、`PutRecord` APIを使って FeatureGroup にデータを追加できるようになります。

この API は高い TPS に対応でき、複数のストリームから呼び出せるように設計されています。
それらの PUT リクエストからのデータはバッファリングされ、チャンクでS3に書き込まれます。

ファイルは取り込まれてから数分以内にオフラインストアに書き込まれます。
取り込み処理を高速化するために、複数のワーカーを指定して同時にジョブを実行させることもできます。

FeatureGroup に単一のレコードを追加するには、`put_record(...)` を使います。

pandas の DataFrame の内容を Feature Store に取り込むには、`ingest(...)` を使用してください。
`max_worker` には、`data_frame` の複数のパーティションで並列に作業するために作成するスレッドの数を設定することができます。

In [ ]:
import pandas as pd

df_records = pd.DataFrame.from_dict(records)
df_records["split_type"] = "train"
df_records

# DataFrame の `Object` を、Feature Store でサポートされるデータ型の `String` にキャストする

In [ ]:
def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        if data_frame.dtypes[label] == "object":
            data_frame[label] = data_frame[label].astype("str").astype("string")

In [ ]:
%%time

cast_object_to_string(df_records)

feature_group.ingest(data_frame=df_records, max_workers=3, wait=True)

# 特徴量ストアがアクティブになるまで待機
## _注意:  数分かかります。少々お待ちください。_

In [ ]:
feature_store_describe_response = feature_group.describe()

while "OfflineStoreStatus" not in feature_store_describe_response.keys():
    feature_store_describe_response = feature_group.describe()
    print("[INFO] Waiting for OfflineStore to be created.")
    # print(json.dumps(feature_store_describe_response, indent=4, sort_keys=True, default=str))
    sleep(60)

print("Offline store created.")

In [ ]:
offline_store_status = None

while offline_store_status != 'Active':
    try:
        offline_store_status = feature_group.describe()['OfflineStoreStatus']['Status']
    except:
        pass
print('Offline store status: {}'.format(offline_store_status))

# 特徴量ストアにクエリ

In [ ]:
feature_store_query = feature_group.athena_query()

feature_store_table = feature_store_query.table_name

query_string = """
    SELECT 
        input_ids,
        input_mask,
        segment_ids, 
        label_id,
        review_id,
        date,
        label,
        split_type
    FROM "{}" 
    WHERE split_type='train' 
    LIMIT 3
""".format(feature_store_table)

print('Glue Catalog table name: {}'.format(feature_store_table))
print('Running query: {}'.format(query_string))

In [ ]:
output_s3_uri = 's3://{}/query_results/{}/'.format(bucket, prefix)
print(output_s3_uri)

In [ ]:
feature_store_query.run(
    query_string=query_string, 
    output_location=output_s3_uri
)

feature_store_query.wait()

In [ ]:
import pandas as pd
pd.set_option("max_colwidth", 100)

df_feature_store = feature_store_query.as_dataframe()
df_feature_store

# 特徴量ストアをレビュー

![Feature Store](img/feature_store_sm_extension.png)

# リソースを解放

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>